In [ ]:
import urllib.request as urllib2
import json
import collections, itertools
import requests
import csv
import requests
import shutil
import os
import re
import numpy as np

from bs4 import BeautifulSoup

In [ ]:
#!pip install recipe_scrapers

In [ ]:
from recipe_scrapers import scrape_me
def save_image_fn(dish, image_url, num):
    #save image to file 
    resp = requests.get(image_url, stream=True)
    if not os.path.exists("data/" + dish):
        os.makedirs("data/" + dish)
    local_file = open("data/" + dish + "/" + str(num) + "_fn.jpg", 'wb')
    resp.raw.decode_content = True
    shutil.copyfileobj(resp.raw, local_file)
    del resp
    return

def main_scrape(recipe_url_dict):
    #Take food types from classes.txt and search AllRecipes for links
    #Pagecount is how many pages to pull, each page has 25 results I think
    global f
    f = open('data/recipes_2.txt', 'w')
    for food_class in recipe_url_dict.keys():
        count = 0
        dish = food_class
        print(dish)
        for rec in recipe_url_dict[dish]:
            scraper = scrape_me(rec)
            try:
                nutrition_d = scraper.nutrients()
                nutrition = []
                for i in nutrition_d:
                    ty = i.replace('Content', '').lower()
                    
                    va = nutrition_d[i].lower().replace('milli', 'm').replace('grams', 'g')
                    if 'calories' in ty:
                        nutrition.append(va)
                    else:
                        nutrition.append(ty+' '+va)
            except:
                print('no_nutr')
                continue
            ingredients = scraper.ingredients()
            try:
                img = scraper.image()
                save_image_fn(dish, img, count)
            except:
                continue
            
            
            result = "|".join([dish, rec, dish + "/" + str(count) + "_fn.jpg", f"rating={'None'}", str(nutrition), str(ingredients)]) + "\n"
            try:
                f.write(result)
                count+=1
            except Exception as e:
                print(e)
                print(result)
                continue
            print(count)
            if count>20:
                break
    
    f.close()

In [ ]:
# f.close()

In [ ]:
# Search Queries for sparsely filled recipes
search_dict = {'beef_tartare': 'https://www.yummly.com/recipes?q=steak%20tartare&sortBy=rel',
'cheese_plate': 'https://www.yummly.com/recipes?q=cheese%20plate&sortBy=rel',
'takoyaki': 'https://www.yummly.com/recipes?q=takoyaki&sortBy=rel',
'beef_carpaccio': 'https://www.yummly.com/recipes?q=carpaccio&sortBy=rel',
'sashimi': 'https://www.yummly.com/recipes?q=sashimi&sortBy=rel',
'bibimbap': 'https://www.yummly.com/recipes?q=bibimbap&sortBy=rel',
'escargots': 'https://www.yummly.com/recipes?q=escargots&sortBy=rel',
'hamburger':    'https://www.yummly.com/recipes?q=hamburger&sortBy=rel',
'fish_and_chips': 'https://www.yummly.com/recipes?q=fish%20and%20chips&sortBy=rel',
'croque_madame': 'https://www.yummly.com/recipes?q=croque%20madame&sortBy=rel',
'macarons': 'https://www.yummly.com/recipes?q=croque%20macarons&sortBy=rel',
'tuna_tartare': 'https://www.yummly.com/recipes?q=tuna%20tartare&sortBy=rel',
'gyoza': 'https://www.yummly.com/recipes?q=gyoza&sortBy=rel',
'seaweed_salad': 'https://www.yummly.com/recipes?q=seaweed%20salad&sortBy=rel',
'club_sandwich': 'https://www.yummly.com/recipes?q=club%20sandwich&sortBy=rel',
'churros': 'https://www.yummly.com/recipes?q=churros&sortBy=rel',
'poutine': 'https://www.yummly.com/recipes?q=poutine&sortBy=rel',}

In [ ]:
rec_url_dict = {}
for i in search_dict:
    url_list = []
    scraper = scrape_me(search_dict[i])
    for x in scraper.links():
        if 'class' in x.keys():
            if 'link-overlay' in x['class']:
                url_list.append('https://www.yummly.com'+x['href'])
    rec_url_dict[i] = url_list

In [ ]:
main_scrape(rec_url_dict)

# After the code above, Manually went through the images in each category and removed images that were not actually part of the class

In [1]:
import os
# os.path.isfile(path)
file1 = open("data/recipes_2.txt","r")
file2 = open('data/recipes_filtered_2.txt', 'w')
for i in file1.readlines():
    img_path = i.split('|')[2]
    nutr = i.split('|')[4]

    if os.path.isfile('data/'+img_path) and len(nutr)>5:
        file2.write(i)
    else:
        pass
file1.close()
file2.close()

In [2]:
file = open("classes.txt", "r")

all_classes = file.readlines()
class_counts = {}
for i in all_classes:
    j = i.replace('\n', '')
    class_counts[j]=0
pics = open("data/recipes_filtered_all.txt", 'r')
all_pics = pics.readlines()
for i in all_pics:
    clas = i.split('|')[0]
    class_counts[clas]+=1
file.close()
pics.close()

In [3]:
import pandas as pd
count_df = pd.DataFrame({'class':class_counts.keys(), 'count':class_counts.values()})
count_df[count_df['count']<20].sort_values(by='count')

,class,count
53,hamburger,5
3,beef_carpaccio,8
17,cheese_plate,9
61,lobster_roll_sandwich,10
6,beignets,10
12,cannoli,11
64,miso_soup,12
97,takoyaki,12
39,foie_gras,12
55,hot_dog,13
